In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Model:  
    """
    This class represents an AI model.
    """
    
    def __init__(self):
        """
        Constructor for Model class.
  
        Parameters
        ----------
        self : object
            The instance of the object passed by Python.
        """
        # TODO: Replace the following code with your own initialization code.
        self.num_classes= 3
        self.model = MyCNN(3)

    #Filling nan values of X with mean of adjacent pixels
    def replace_nan_with_adjacent_mean(self, img):
        n_channel,n_row,n_col = img.shape
        for channel in range(n_channel):
            for row in range(n_row):
                for col in range(n_col):
                    if np.isnan(img[channel, row, col]):
                        adjacent = []
                        for i in range(row-1, row+1):
                            for j in range(col-1, col+1):
                                if 0 <= i < n_row and 0 <= j < n_col and not np.isnan(img[channel, i, j]):
                                    adjacent.append(img[channel, i, j])
                        if adjacent:
                            img[channel, row, col] = np.mean(adjacent)
                        else:
                            img[channel, row, col] = 0
    
    
    def fit(self, X, y):
        """
        Train the model using the input data.
        
        Parameters
        ----------
        X : ndarray of shape (n_samples, channel, height, width)
            Training data.
        y : ndarray of shape (n_samples,)
            Target values.
            
        Returns
        -------
        self : object
            Returns an instance of the trained model.
        """
        
        
        #Clip values from range 0 to 255
        X = np.clip(X, 0, 255)
        
        #Filling nan values of X with mean of adjacent pixels
        for i in range(X.shape[0]):
            self.replace_nan_with_adjacent_mean(X[i])
        
        #Flatten X and change to df
        flattened_images = X.reshape(X.shape[0], -1)
        images_df = pd.DataFrame(flattened_images)
        labels_df = pd.DataFrame(y)
        
        #Remove y rows with nan values
        labels_row_with_nan = labels_df.dropna().index
        images_df_filtered = images_df.loc[labels_row_with_nan]
        labels_df_filtered = labels_df.loc[labels_row_with_nan]
        
        #Oversampling minority class
        majority_n = labels_df_filtered[0].value_counts().max()
        data_df = pd.concat([images_df_filtered, labels_df_filtered], axis=1)
        majority_index = data_df.iloc[:,768].value_counts().idxmax() 
        
        #Separate majority class and the rest
        majority_data = data_df[data_df.iloc[:,768] == majority_index]
        minority_data = data_df[data_df.iloc[:,768] != majority_index]

        oversampled_minority_data = pd.DataFrame()
        #For each minority class, oversample the class with the majority class count
        for minority_class in minority_data.iloc[:,768].unique():
            minority_class_data = minority_data[minority_data.iloc[:,768] == minority_class]
            oversampled_minority_class_data = minority_class_data.sample(n=majority_n, replace=True)
            oversampled_minority_data = pd.concat([oversampled_minority_data, oversampled_minority_class_data], axis=0)
        
        #Merge all the data together
        oversampled_data = pd.concat([majority_data, oversampled_minority_data], axis=0)

        images_balanced = oversampled_data.iloc[:, :oversampled_data.shape[1]-1]
        labels_balanced = oversampled_data.iloc[:, -1]    
        
        #Change data to numpy
        images_balanced = images_balanced.values.reshape(-1, 3, 16, 16)
        labels_balanced = labels_balanced.to_numpy()
        
        
        batch_size = 128
        #Change dataset to tensor and put in dataloader
        image_tensor = torch.tensor(images_balanced, dtype=torch.float32)
        label_tensor = torch.tensor(labels_balanced, dtype=torch.long)
        training_set = TensorDataset(image_tensor, label_tensor)
        training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)

        #Initialise optimizer and loss function
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)

        #Train the model for set no. of epochs
        num_epochs = 20  
        for epoch in range(num_epochs):
            self.model.train()
            for inputs, labels in training_loader:
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = loss_fn(outputs, labels)
                loss.backward()
                optimizer.step()
        
        return self
    
    def predict(self, X):
        """
        Use the trained model to make predictions.
        
        Parameters
        ----------
        X : ndarray of shape (n_samples, channel, height, width)
            Input data.
            
        Returns
        -------
        ndarray of shape (n_samples,)
        Predicted target values per element in X.
           
        """
        # TODO: Replace the following code with your own prediction code.
        
        #Clip values that are out of range
        X = np.clip(X, 0, 255)
        
        #Fill in nan values of X
        for i in range(X.shape[0]):
            self.replace_nan_with_adjacent_mean(X[i])
            
        #Change X to tensor and run it with the model
        images_tensor = torch.tensor(X, dtype=torch.float32)
        predictions = self.model(images_tensor)
        
        #Get the class with the highest probability
        predictions = torch.argmax(predictions, dim=1).numpy()
        
        return predictions  

    
class MyCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1),
            nn.MaxPool2d(kernel_size=2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(1600, 512)
        self.linear2 = nn.Linear(512, 128)
        self.linear3 = nn.Linear(128, num_classes)
        self.leaky = nn.LeakyReLU()
        self.DO = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.conv1(x)
        x = self.leaky(x)
        x = self.conv2(x)
        x = self.leaky(x)
        x = self.DO(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.leaky(x)
        x = self.linear2(x)
        x = self.leaky(x)
        x = self.linear3(x)
        return x

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# Load data
with open('data.npy', 'rb') as f:
    data = np.load(f, allow_pickle=True).item()
    X = data['image']
    y = data['label']

In [ ]:
import time
for i in range(10):
    # Split train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

    # Filter test data that contains no labels
    # In Coursemology, the test data is guaranteed to have labels
    nan_indices = np.argwhere(np.isnan(y_test)).squeeze()
    mask = np.ones(y_test.shape, bool)
    mask[nan_indices] = False
    X_test = X_test[mask]
    y_test = y_test[mask]


    start_time = time.time()
    model = Model()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluate model predition
    # Learn more: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics
    print("F1 Score (macro): {0:.2f}".format(f1_score(y_test, y_pred, average='macro'))) # You may encounter errors, you are expected to figure out what's the issue.
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.6f} seconds")